In [141]:
from neo4j import GraphDatabase
import re
import pandas as pd
import itertools
import numpy as np
import json
from datetime import datetime
import networkx as nx 
from networkx.readwrite import json_graph
import matplotlib.pyplot as plt 
from io import StringIO
from elasticsearch import Elasticsearch, helpers
from Script.helper_v2 import *
from Script.generateHtml_v4 import generateHtml

## Read df with score

In [142]:
csvFile = "D:/Users/figohjs/Documents/Network/data/interim/2020-09-03_482ID_SubGraphDF.csv"

df = pd.read_csv(csvFile)

#convert str to list - export issue 
def colBeList(colArray, colName):
    result = []
    for colList in [i.replace("{", '').replace('}', '')\
                     .replace("[", '').replace(']', '')\
                    .replace("\'", '').split(',') for i in colArray]:
        if colName in ['NeoIds', 'TransIds', 'targetMuleAccountNeoIds']:
            result.append([int(i.strip()) for i in colList])
        else:
            result.append([i.strip() for i in colList])
    return result

colList = ['NeoIds', 'TransIds', 'targetMuleAccountID', 'targetMuleAccountNeoIds']
for col in colList:
    if col in df.columns:
        df[col] = colBeList(df[col].values, col)

In [143]:
df2 = pd.DataFrame(df.sort_values('Score', ascending = False).iloc[0]).T

idList = df2['targetMuleAccountID'].values[0]

## Validation

In [34]:
jsonFile = "D:/Users/figohjs/Documents/Network/data/interim/2020-08-26_482IDAddress_SubNet.json"
with open(jsonFile, 'r') as f:
    tempGraphDict = json.load(f)

In [24]:
#from 4 hops using one central node
jsonFile = "D:/Users/figohjs/Documents/Network/data/interim/2020-09-21_topNetworkDC.json"
with open(jsonFile, 'r') as f:
    tempGraphDict4Hops = json.load(f)

In [25]:
idList4Hops = [i['pIdList'] for i in tempGraphDict['nodes'] if i['label'] == 'Entity']
idList4Hops = [j for i in idList4Hops for j in i]
len(idList4Hops)

1217

In [3]:
#from yan ling
file = "D:/Users/figohjs/Documents/Network/data/interim/id_mule.xlsx"
dfIdYL = pd.read_excel(file)
idListYL = [i[0] for i in dfIdYL.values]

In [23]:
len(idListYL)

248

In [12]:
#from jiun
idList = []
fileName = "D:/Users/figohjs/Documents/Network/data/interim/idListTopNetwork.txt"
with open(fileName, "r") as f:
    for i in f.readlines():
        idList.append(re.sub('\n','',i))

In [22]:
#id which cannot be found from yan
[i for i in idList if i not in idListYL]

['790405135157',
 '910501136137',
 '930924135177',
 '840103135672',
 '960430136767']

In [26]:
[i for i in idList if i not in idList4Hops]

[]

In [29]:
idCannotFound = ['790405135157',
 '910501136137',
 '930924135177',
 '840103135672',
 '960430136767']
nodeInfo = [i for i in tempGraphDict['nodes'] if i['label'] == 'Entity']
[i for i in nodeInfo if len(set(idCannotFound) & set(i['pIdList']))>0]

[{'uid': 'e-7652088d-a3f0-44a6-a34f-1b7c505c23bf',
  'pOccupation': ['Businessman / Businesswoman',
   'Mechanic',
   'Salesman / Salesgirl',
   'Chef / Cook / Baker',
   'Plumber',
   'Others'],
  'pName': 'HAZMAN BIN PUTIT',
  'pNameList': ['HAZMAN BIN PUTIT', 'HAZMAN BIN PUTT'],
  'pCountry': ['MALAYSIA'],
  'pIdList': ['790405135157'],
  'contact_list': ['0128671321',
   'HAZMAN7945@GMAIL.COM',
   '0192701507',
   '0128631321'],
  'pOccupationDesc': ['STALL AND MARKET SALESPERSONS',
   'ELECTRICAL MECHANICS AND FITTERS',
   'SALES WORKER',
   'PLUMBER AND PIPE FITTER'],
  'label': 'Entity',
  'NeoId': 9998219},
 {'uid': 'e-10297bfc-9d35-4991-b53a-e420601172ea',
  'pOccupation': ['Driver', 'Salesman / Salesgirl', 'Mechanic', 'Clerk'],
  'pName': 'PILATUS SAMSON',
  'pNameList': ['PILATUS SAMSON'],
  'pCountry': ['MALAYSIA'],
  'pIdList': ['960430136767'],
  'contact_list': ['0182653791',
   '085421010',
   '01169308651',
   'CHRISTCLOE6745@GMAIL.COM',
   '01160829673'],
  'pOccupati

## Consolidate network

In [30]:
#from 4 hops using one central node
jsonFile = "D:/Users/figohjs/Documents/Network/data/interim/2020-09-21_topNetworkDC.json"
with open(jsonFile, 'r') as f:
    tempGraphDict4Hops = json.load(f)

In [37]:
len(tempGraphDict4Hops['links'])

1012

In [78]:
#network with most scores
dfSubset = df.sort_values('Score', ascending = False)[:1]

#convert data format to json
resultGraph = getNetworkJson(dfSubset, tempGraphDict)

In [36]:
len(resultGraph['links'])

494

In [40]:
dfSubset['TransIds'].values[0]

[2324662,
 2324663,
 3358295,
 3358296,
 3364874,
 5889540,
 5889541,
 6550302,
 6550300,
 7313279,
 586499,
 7372793,
 7372791,
 8211536,
 8211535,
 8211638,
 8211640,
 8213386,
 8213385,
 8218078,
 8218080,
 8218613,
 8218614,
 8254915,
 8254917,
 8432090,
 23799130,
 18506708,
 22825411,
 23667828,
 23667826,
 23799031,
 23799033,
 23799129,
 23799131,
 23822925,
 23822927,
 23822970,
 23822972,
 23823004,
 23823005,
 23823440,
 23823438,
 23888308,
 23888307,
 23891793,
 23891792,
 23924555,
 23924553,
 23924560,
 23924558,
 23925220,
 23925222,
 23925312,
 23925310,
 23925371,
 23925373,
 23925376,
 23925378,
 23925416,
 23925417,
 23925422,
 23925420,
 23926080,
 23926082,
 23942187,
 23942186,
 23944937,
 23944935,
 23945042,
 23945041,
 23946478,
 23946476,
 23946961,
 23946960,
 24024034,
 24024036,
 24024041,
 24024043,
 24024123,
 24024125,
 24024127,
 24024129,
 24024134,
 24024136,
 24024138,
 24024140,
 24024142,
 24024144,
 24024150,
 24024152,
 24024197,
 24024198,
 240

In [114]:
contactNodes = [i['target'] for i in tempGraphDict4Hops['links'] if i['type'] == 'HAS_CONTACT']
#all neo4j Id from unwanted contact nodes
removeContactNodes = [i for i,j in dict(pd.Series(contactNodes).value_counts()).items() if j==1]

#consolidate links from 4 hops and top network
linkList = []
for link in tempGraphDict4Hops['links']:
    if link['target'] not in removeContactNodes:
        linkList.append(link)
        
finalLinkList = linkList 
# finalLinkList = linkList

In [115]:
len(finalLinkList)

881

In [117]:
#consolidate nodes
nodeList = []
for node in tempGraphDict4Hops['nodes']:
    if node['NeoId'] not in removeContactNodes:
        nodeList.append(node)

finalNodeList = nodeList

In [118]:
len(finalNodeList)

637

In [119]:
finalGraphDict = {}
finalGraphDict['nodes'] = finalNodeList
finalGraphDict['links'] = finalLinkList

In [121]:
def changeTargetNodeLabel(graph, targetNeoIdList):
#wrangle json so that target node is diff color
#replace label of entity with target
    for no, nodeDict in enumerate(graph['nodes']):
        if nodeDict['label'] == 'Entity':
            if nodeDict['NeoId'] in targetNeoIdList:
                graph['nodes'][no]['label'] = 'Target'
                
changeTargetNodeLabel(finalGraphDict, [10112866])

In [122]:
#save json in a html format
htmlFile = "D:/Users/figohjs/Documents/Network/Html/networkMostScore4Hops.html"
generateHtml(finalGraphDict, htmlFile)

D:/Users/figohjs/Documents/Network/Html/networkMostScore4Hops.html is saved


In [133]:
[i for i in resultGraph['nodes'] if i['label'] == 'STR']

[{'uid': 'r-526db021-c265-43ff-8050-4bbcebfe5b2e',
  'pOffence': 'Money Laundering',
  'pStateId': 'SARAWAK',
  'pTrxAmount': '0.0',
  'pReportDate': '07/10/2019',
  'pRecordTypeID': 'STR financial',
  'pAttempted': 'N',
  'pToTrxDate': '29/09/2019',
  'pBranchId': 'KUCHING - JLN HAJI TAHA',
  'pRecordID': 'AA/004/S/2019/005219',
  'pFromTrxDate': '30/03/2018',
  'label': 'STR',
  'NeoId': 14425790},
 {'uid': 'r-7ef27a79-4b0d-4c63-9f81-2bdc5dd3b4a7',
  'pOffence': 'Fraud',
  'pStateId': 'SARAWAK',
  'pTrxAmount': '27550.0',
  'pReportDate': '12/09/2019',
  'pRecordTypeID': 'STR financial',
  'pAttempted': 'N',
  'pToTrxDate': '12/08/2019',
  'pBranchId': 'TAMAN BDC, KUCHING',
  'pRecordID': 'AA/010/S/2019/019858',
  'pFromTrxDate': '08/08/2019',
  'label': 'STR',
  'NeoId': 14480177},
 {'uid': 'r-a9cb4599-1cdb-4a58-81f5-3ab25db44c11',
  'pOffence': 'Fraud',
  'pStateId': 'SARAWAK',
  'pTrxAmount': '49221.14',
  'pReportDate': '24/09/2019',
  'pRecordTypeID': 'STR financial',
  'pAttemp

## Consolidate STR

In [123]:
strFileName = "D:/Users/figohjs/Documents/Network/data/interim/2020-09-04_StrTopNetwork.csv"
dfStr = pd.read_csv(strFileName)

In [126]:
dfStr.columns

Index(['record_id', 'name_list', 'id_list', 'contact_list', 'summary',
       'detail', 'reported_date', 'trx_date', 'account', 'suspicion_desc',
       'reactive', 'uid'],
      dtype='object')

In [129]:
len(dfStr['record_id'].values)

149

In [124]:
strFileName = "D:/Users/figohjs/Documents/Network/data/final/str_mule.xlsx"
dfStr2 = pd.read_excel(strFileName)

In [132]:
dfStr2.columns

Index(['RECORD_ID', 'RECORD_TYPE_NAME', 'TRX_AMOUNT', 'SUBMITTED_DATE',
       'ENTITY_TYPE', 'ENTITY_NAME', 'NEW_IC', 'OLD_IC',
       'OTHER_IDENTIFICATION_NO', 'OTHER_IDENTIFICATION_TYPE', 'PASSPORT_NO',
       'BUSINESS_REG_NO', 'SUSPICION_DESC', 'GROUND_SUSPICION',
       'OFFENCE_TYPE_DESC', 'OTHER_SUSPICION', 'TAGGED'],
      dtype='object')

In [138]:
strFileName = "D:/Users/figohjs/Documents/Network/data/final/mule_str2.xlsx"
dfStr3 = pd.read_excel(strFileName)

In [139]:
dfStr3.shape

(811, 16)

In [140]:
dfStr3.columns

Index(['RECORD_ID', 'RECORD_TYPE_NAME', 'TRX_AMOUNT', 'REPORTED_DATE',
       'ENTITY_TYPE_DESC', 'ENTITY_NAME', 'NEW_IC', 'OLD_IC',
       'OTHER_IDENTIFICATION_NO', 'OTHER_IDENTIFICATION_TYPE', 'PASSPORT_NO',
       'BUSINESS_REG_NO', 'SUSPICION_DESC', 'GROUND_SUSPICION',
       'OFFENCE_TYPE_DESC', 'OTHER_SUSPICION'],
      dtype='object')

In [150]:
#check if all targets can be found in dfStr3
len(idList), len(set(idList) & set(icList))

(23, 23)

In [152]:
commonList = set(idList) & set(icList)
dfStr3['NEW_IC2'] = [re.sub('-','',str(i)) for i in dfStr3['NEW_IC']]
dfStr3_subset = dfStr3.query('NEW_IC2 in @commonList')

In [153]:
dfStr3_subset.columns

Index(['RECORD_ID', 'RECORD_TYPE_NAME', 'TRX_AMOUNT', 'REPORTED_DATE',
       'ENTITY_TYPE_DESC', 'ENTITY_NAME', 'NEW_IC', 'OLD_IC',
       'OTHER_IDENTIFICATION_NO', 'OTHER_IDENTIFICATION_TYPE', 'PASSPORT_NO',
       'BUSINESS_REG_NO', 'SUSPICION_DESC', 'GROUND_SUSPICION',
       'OFFENCE_TYPE_DESC', 'OTHER_SUSPICION', 'NEW_IC2'],
      dtype='object')

In [158]:
#number of str by target
dfFinal = dfStr3_subset.groupby('NEW_IC')['RECORD_ID'].count().reset_index()
dfFinal.rename(columns = {'RECORD_ID':'NumberRecord'}, inplace = True)

In [159]:
filename = "D:/Users/figohjs/Documents/Network/data/final/strCountByTarget.xlsx"
dfFinal.to_excel(filename, index = False)

## Extract network with 4 hops

In [4]:
url = "bolt://localhost:7687"
user = 'neo4j'
password = 'bnm@1234' 
driver = GraphDatabase.driver(url, auth=(user, password))

In [5]:
def runNeo4j(query, p = True):
    resultList = []
    with driver.session() as session:
        res = session.run(query)
        if p:
            for i in res:
                resultList.append(i)
                print(i[0])
        else:
            for i in res:
                resultList.append(i)
            return resultList

In [ ]:
#4 hops
hops = 4

start = datetime.now()
query1 = "WITH " + str(idList) + ' as idList '
query2 = "MATCH (a)-[r*..%s]->(b) "%hops
query3 = "WHERE any(i in a.pIdList where i in idList) or any(i in b.pIdList where i in idList)"
query4 = "RETURN a, r, b"
finalQuery = query1 + query2 + query3 + query4
result = runNeo4j(finalQuery, p = False)
time = datetime.now() - start
print("Estimated time: %s"%time)

In [6]:
jsonFile = "D:/Users/figohjs/Documents/Network/data/interim/2020-08-04_482ID_SubNet.json"
with open(jsonFile, 'r') as f:
    tempGraphDict = json.load(f)

#convert data format to json
resultGraph = getNetworkJson(df2, tempGraphDict)

In [7]:
recordIdList = [i['pRecordID'] for i in resultGraph['nodes'] if i['label'] == 'STR']

accountList = [i['pAccountNo'] for i in resultGraph['nodes'] if i['label'] == 'Account']

nameList = [i['pName'] for i in resultGraph['nodes'] if i['label'] in ['Entity', 'Target']]

In [18]:
idList = [i['pIdList'] for i in resultGraph['nodes'] if i['label'] in ['Entity', 'Target']]
idList = [j for i in idList for j in i]

In [19]:
#write into text
fileName = "D:/Users/figohjs/Documents/Network/data/interim/idListTopNetwork.txt"
with open(fileName, 'w') as filehandle:
    for idNo in idList:
        filehandle.write('%s\n' %idNo)

In [24]:
#write into text
fileName = "D:/Users/figohjs/Documents/Network/data/interim/nameListTopNetwork.txt"
with open(fileName, 'w') as filehandle:
    for name in nameList:
        filehandle.write('%s\n' %name)

In [20]:
len(nameList)

45

## Details extraction from es

In [8]:
#connect es
es = Elasticsearch([{'host': 'localhost', 'port':9200}])

In [9]:
def multiSearch(fieldName, valueList):
    #record head and body msg for request
    searchBulk = StringIO()
    for value in valueList:
        #head
        headMsg = {'index': indexName}
        searchBulk.write('%s \n' %json.dumps(headMsg))
        #body
        bodyMsg = {"query": {'match':{fieldName:value}}}
        searchBulk.write('%s \n' %json.dumps(bodyMsg))
    
    #start multiSearch
    res = es.msearch(index = indexName, body = searchBulk.getvalue())
    
    #extract hits value
    result = [i['hits']['hits'] for i in res['responses']]
    
    return result

In [10]:
#get all indices in es
sorted(list(es.indices.get_alias('*').keys()))

['.apm-agent-configuration',
 '.kibana_1',
 '.kibana_task_manager_1',
 '.security-7',
 'account_db',
 'assignedreadonly',
 'audit_trail',
 'contact_db',
 'detail_ctr_2013',
 'detail_ctr_2014',
 'detail_ctr_2015',
 'detail_ctr_2016',
 'detail_ctr_2017',
 'detail_ctr_2018',
 'detail_ctr_2019',
 'detail_ctr_2020',
 'detail_itis',
 'detail_str',
 'distdates',
 'distratio2020',
 'distrecords',
 'entity_db',
 'itis_db',
 'leave',
 'removedrecord',
 'report_db',
 'tax']

### str

In [50]:
indexName = 'detail_str'
queryAll = {'match_all':{}}
res = es.search(index = indexName, size = 3, body = {'query':queryAll})
print(res['hits']['hits'][2])

{'_index': 'detail_str', '_type': '_doc', '_id': 'c7E9enABgj0HNtLN1IDm', '_score': 1.0, '_source': {'record_id': '117/271800V/S/2015/000001F', 'name_list': ['ERNEST CHUKWUDI AGUPUSI'], 'id_list': ['AA878490', 'AA00732712'], 'contact_list': [], 'summary': {'Reported Date': '28/11/2013', 'Org Name': 'Spectrum Forex Sdn. Bhd.', 'Report Id': '117/271800V/S/2015/000001F', 'Attempted': 'N', 'Offence': 'Unknown', 'Reactive': 'No', 'Role': {'ERNEST CHUKWUDI AGUPUSI': ['Customer']}, 'Account Holder': ['ERNEST CHUKWUDI AGUPUSI']}, 'detail': {'report': {'ORG_ID': 'Spectrum Forex Sdn. Bhd.', 'RECORD_ID': '117/271800V/S/2015/000001F', 'RECORD_TYPE_ID': 'STR Money changer', 'ATTEMPTED_FLAG': 'N', 'STATE_ID': 'UNKNOWN', 'BRANCH_ID': 'UNKNOWN', 'ACCOUNT_TYPE_ID': 'UNKNOWN', 'FROM_TRX_DATE': '01/09/2013', 'TO_TRX_DATE': '28/11/2013', 'TRX_AMOUNT': '861744.0', 'CURRENCY_ID1': 'Malaysian Ringgit', 'CURRENCY_ID2': 'US Dollar', 'REPORTED_DATE': '28/11/2013', 'GROUND_SUSPICION': 'Business', 'OFFENCE_TYPE_ID

In [80]:
#query es
indexName = 'detail_str'
strDetailResult = multiSearch('record_id', recordIdList)
len([i for i in strDetailResult if len(i) !=0])

149

In [105]:
strList = []
for record in strDetailResult:
    strList.append(record[0]['_source'])
len(strList)

149

In [112]:
dfStr = pd.DataFrame(strList)
strFileName = "D:/Users/figohjs/Documents/Network/data/interim/2020-09-04_StrTopNetwork.csv"
dfStr.to_csv(strFileName, index = False)

### itis

In [88]:
indexName = 'detail_itis'
queryAll = {'match_all':{}}
res = es.search(index = indexName, size = 3, body = {'query':queryAll})
print(res['hits']['hits'][1])

{'_index': 'detail_itis', '_type': '_doc', '_id': 'Ul1qYHABpspFooR7FoE3', '_score': 1.0, '_source': {'reported_date': '13/08/2012', 'id_list': ['210262T'], 'name_list': ['ABB MALAYSIA SDN BHD', 'ABB CAPITAL BV ZURICH BRANCH SWITZERLAND'], 'detail': {'RESIDENT_NAME': 'ABB MALAYSIA SDN BHD', 'RESIDENT_ID': '210262T', 'NON_RESIDENT_COUNTRY': 'SWITZERLAND', 'NON_RESIDENT_NAME': 'ABB CAPITAL BV ZURICH BRANCH SWITZERLAND', 'REPORTED_DATE': '13/08/2012', 'ORG_ID': 'DEUTSCHE BANK (MALAYSIA) BERHAD', 'TRANSACTION_TYPE': 'Payments', 'AMOUNT': '2198217.0', 'TRANSACTION_PURPOSE': 'MACHINERY, NON-CUSTOMISED PACKAGED SOFTWARE AND TRANSPORT EQUIPMENT'}, 'uid': ['e-b58ccfc2-88fc-4fac-84a2-7f570b69d6c3', 'e-3722ccb4-292a-4f51-b87d-65939b995113']}}


In [115]:
#query es
indexName = 'detail_itis'
itisResult = multiSearch('RESIDENT_NAME', nameList)
len([i for i in itisResult if len(i) !=0])

0

In [116]:
#query es
indexName = 'detail_itis'
itisResult = multiSearch('NON_RESIDENT_NAME', nameList)
len([i for i in itisResult if len(i) !=0])

0

### ctr

In [57]:
indexName = 'detail_ctr_2013'
queryAll = {'match_all':{}}
res = es.search(index = indexName, size = 3, body = {'query':queryAll})
print(res['hits']['hits'][0])

{'_index': 'detail_ctr_2013', '_type': '_doc', '_id': 'lOSEYHABpspFooR7qIZR', '_score': 1.0, '_source': {'record_id': 'AA/014/C/2013/045221U', 'name_list': ['SETO LIAN WAH'], 'id_list': ['630113015679', '7007853'], 'contact_list': [], 'summary': {'Year': '2013', 'Trx Amount': '88500.0', 'Org Name': 'Hong Leong Bank Berhad', 'Account No': '16750132071', 'Account Type': 'UNKNOWN', 'Role': {'SETO LIAN WAH': ['Account Holder']}, 'Account Holder': ['SETO LIAN WAH']}, 'detail': {'report': {'ORG_ID': 'Hong Leong Bank Berhad', 'RECORD_ID': 'AA/014/C/2013/045221U', 'STATE_ID': 'UNKNOWN', 'BRANCH_ID': 'UNKNOWN', 'ACCOUNT_NO': '16750132071', 'ACCOUNT_TYPE_ID': 'UNKNOWN', 'FROM_TRX_DATE': '22/03/2013', 'TRX_AMOUNT': '88500.0', 'REPORTED_DATE': '24/03/2013'}, 'entity': [{'SEQ_NO': '4', 'ENTITY_ID': '349526261', 'ENTITY_TYPE': 'Account Holder', 'ENTITY_INDICATOR': 'Individual', 'ENTITY_NAME': 'SETO LIAN WAH', 'NEW_IC': '630113015679', 'OLD_IC': '7007853', 'GENDER': 'Unknown', 'COUNTRY_ID': 'MALAYSIA

In [72]:
#query es
indexName = 'detail_ctr_2013'
ctrDetail2013Result = multiSearch('record_id', recordIdList)
len([i for i in ctrDetail2013Result if len(i) !=0])

0

In [73]:
#query es
indexName = 'detail_ctr_2014'
ctrDetail2014Result = multiSearch('record_id', recordIdList)
len([i for i in ctrDetail2014Result if len(i) !=0])

0

In [74]:
#query es
indexName = 'detail_ctr_2015'
ctrDetail2015Result = multiSearch('record_id', recordIdList)
len([i for i in ctrDetail2015Result if len(i) !=0])

0

In [84]:
#query es - account no of sender
indexName = 'detail_ctr_2015'
ctrDetail2015Result = multiSearch('Account No', accountList)
len([i for i in ctrDetail2015Result if len(i) !=0])

0

In [117]:
#query es - account no of recipient
indexName = 'detail_ctr_2015'
ctrDetail2015Result = multiSearch('account', accountList)
len([i for i in ctrDetail2015Result if len(i) !=0])

1

In [118]:
ctrDetail2015Result

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [{'_index': 'detail_ctr_2015',
   '_type': '_doc',
   '_id': 'oHuaYHABpspFooR7ixKe',
   '_score': 14.894215,
   '_source': {'record_id': 'AA/020/C/2015/265923U',
    'name_list': ['WONG PAK BURN',
     'QL MARINE PRODUCTS SDN BHD',
     'HENWAN SDN BHD'],
    'id_list': ['720415135157', '378679K'],
    'contact_list': ['0198857341'],
    'summary': {'Year': '2015',
     'Trx Amount': '62392.0',
     'Org Name': 'Public Bank Berhad',
     'Account No': 'BBNH032507053',
     'Account Type': 'OTHERS',
     'Role': {'WONG PAK BURN': ['Account Holder', 'PCT'],
      'QL MARINE PRODUCTS SDN BHD': ['Beneficiary'],
      'HENWAN SDN BHD': ['Beneficiary']},
     'Account Holder': ['WONG PAK BURN']},
    'detail': {'report': {'ORG_ID': 'Public Bank Berhad',
      'RECORD_ID': 'AA/020/C/2015/265923U',
      'STATE_ID': 'SARAWAK',
      'BRANCH_ID': 'BATU NIAH',
      'ACCOUNT_NO': 'BBNH032507053',
      'ACCOUNT_TYPE_ID': 'OTH

In [75]:
#query es
indexName = 'detail_ctr_2016'
ctrDetail2016Result = multiSearch('record_id', recordIdList)
len([i for i in ctrDetail2016Result if len(i) !=0])

0

In [76]:
#query es
indexName = 'detail_ctr_2017'
ctrDetail2017Result = multiSearch('record_id', recordIdList)
len([i for i in ctrDetail2017Result if len(i) !=0])

0

In [77]:
#query es
indexName = 'detail_ctr_2018'
ctrDetail2018Result = multiSearch('record_id', recordIdList)
len([i for i in ctrDetail2018Result if len(i) !=0])

0

In [82]:
#query es
indexName = 'detail_ctr_2019'
ctrDetail2019Result = multiSearch('record_id', recordIdList)
len([i for i in ctrDetail2019Result if len(i) !=0])

0

In [83]:
#query es
indexName = 'detail_ctr_2020'
ctrDetail2020Result = multiSearch('record_id', recordIdList)
len([i for i in ctrDetail2020Result if len(i) !=0])

0

In [11]:
ctrList = []
colDict = {'record_id':recordIdList, 'Account No': accountList, 'account':accountList}
for year in range(2013, 2020 + 1):
    indexName = 'detail_ctr_' + str(year)
    for col in colDict.keys():
        ctrResult = multiSearch(col, colDict[col])
        #filter out empty result
        ctrResult = [i for i in ctrResult if len(i) !=0]
        if len(ctrResult) != 0:
            for record in ctrResult:
                ctrList.append(record[0]['_source'])

In [25]:
pd.DataFrame(ctrList).columns

Index(['record_id', 'name_list', 'id_list', 'contact_list', 'summary',
       'detail', 'reported_date', 'trx_date', 'account', 'uid'],
      dtype='object')